BLG 453E  - Homework 2 <br>
Muhammed Tolga Cangöz - 150130024

In [ ]:
import pickle
import matplotlib.pyplot as plt
import torch as th
import cv2
import numpy as np
import moviepy.editor as mpy


with open("stylegan3-t-ffhq-1024x1024.pkl", "rb") as f:
    G = pickle.load(f)['G_ema'].cuda()

G.eval()

for param in G.parameters():
    param.requires_grad = False

In [ ]:
G.synthesis.num_fp16_res = 0

for name, layer in G.synthesis.named_modules():
    if hasattr(layer, 'conv_clamp'):
      layer.conv_clamp = None
      layer.use_fp16 = False

In [ ]:
z1 = th.randn([1, G.z_dim]).cuda()

In [ ]:
z2 = th.randn([1, G.z_dim]).cuda()

In [ ]:
img1 = G(z1, 0).cpu().numpy().squeeze()

In [ ]:
img2 = G(z2, 0).cpu().numpy().squeeze()

In [ ]:
img1 = np.transpose(img1, (1, 2, 0))
img2 = np.transpose(img2, (1, 2, 0))

In [ ]:
img1[img1 > 1] = 1
img1[img1 < -1] = -1
img1 = 255*(img1 + 1) / 2

cv2.imwrite("test1.png", img1[:,:,[2,1,0]])

In [ ]:
img2[img2 > 1] = 1
img2[img2 < -1] = -1
img2 = 255*(img2 + 1) / 2

cv2.imwrite("test2.png", img2[:,:,[2,1,0]])

In [ ]:
images_list = []
for i in range(101):
    z3 = ((100-i)/100)*z1 + (i/100)*z2
    img3 = G(z3, 0).cpu().numpy().squeeze()
    img3 = np.transpose(img3, (1, 2, 0))
    img3[img3 > 1] = 1
    img3[img3 < -1] = -1
    img3 = 255*(img3 + 1) / 2
    images_list.append(img3[:,:,[2,1,0]].astype(np.uint8))

In [ ]:
#import pickle

#with open("images_list.txt", "wb") as fp:
#    pickle.dump(images_list, fp)

#with open("images_list.txt", "rb") as fp:
#    b = pickle.load(fp)

In [ ]:
clip = mpy.ImageSequenceClip(images_list, fps=10)
clip.write_videofile('part2_video.mp4')